In [25]:
import pandas as pd
orders = pd.read_csv(r"C:\Users\sneha yadav\Downloads\orders.csv")
print(orders.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [26]:
users = pd.read_json(r"C:\Users\sneha yadav\Downloads\users.json")
print(users.head())

   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [27]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("restaurants.db")
with open(r"C:\Users\sneha yadav\Downloads\restaurants.sql", "r") as f:sql_script = f.read()
conn.executescript(sql_script)
restaurants = pd.read_sql("SELECT*FROM restaurants", conn)
print(restaurants.head())

OperationalError: table restaurants already exists

In [28]:
merged_1 = pd.merge(orders, users, on= "user_id", how = "left")
print(merged_1.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name       name       city membership  
0               New Foods Chinese  User_2508  Hyderabad    Regular  
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular  
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold  
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold  
4       Royal Eatery South Indian  User_1064       Pune    Regular  


In [29]:
final_df = pd.merge( merged_1, restaurants, on="restaurant_id", how="left")
print(final_df.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

In [30]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [39]:
df= pd.read_csv("final_food_delivery_dataset.csv").head()

In [40]:
df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False)

city
Bangalore    1155.97
Chennai       163.93
Name: total_amount, dtype: float64

In [41]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

cuisine
Italian    1321.91
Chinese    1155.97
Indian      546.68
Mexican     503.45
Name: total_amount, dtype: float64

In [46]:
user_spend = df.groupby("user_id")["total_amount"].sum()
(user_spend > 1000).sum()

np.int64(2)

In [59]:
# Step 1: Define bins & labels (same cell me)
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

# Step 2: Create rating range column
df["rating_range"] = pd.cut(
    df["rating"],
    bins=bins,
    labels=labels
)

# Step 3: Revenue by rating range
df.groupby("rating_range")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


C:\Users\sneha yadav\AppData\Local\Temp\ipykernel_3256\2528119362.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"] \


rating_range
4.6–5.0    1155.97
3.0–3.5     842.97
4.1–4.5     546.68
3.6–4.0     163.93
Name: total_amount, dtype: float64

In [53]:
 df[df["membership"]=="Gold"] \
  .groupby("city")["total_amount"] \
  .mean() \
  .sort_values(ascending=False)


city
Bangalore    1155.97
Chennai       163.93
Name: total_amount, dtype: float64

In [58]:
df.groupby("cuisine").agg(
    restaurant_count=("restaurant_name_x","nunique"),
    revenue=("total_amount","sum")
).sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,1,1155.97
Indian,1,546.68
Italian,1,1321.91
Mexican,2,1006.90


In [60]:
total_orders = len(df)
gold_orders = len(df[df["membership"]=="Gold"])

percentage = round((gold_orders / total_orders) * 100)
percentage


40

In [62]:
restaurant_stats = df.groupby("restaurant_name_x").agg(
    avg_order=("total_amount","mean"),
    total_orders=("order_id","count")
)

restaurant_stats[restaurant_stats["total_orders"] < 20] \
    .sort_values("avg_order", ascending=False) \
    .head()


,avg_order,total_orders
restaurant_name_x,,
Royal Eatery South Indian,1321.91,1
Darbar Kitchen Non-Veg,1155.97,1
New Foods Chinese,842.97,1
Ruchi Curry House Multicuisine,546.68,1
Spice Kitchen Punjabi,163.93,1


In [63]:
df.groupby(["membership","cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Italian    1321.91
Gold        Chinese    1155.97
Regular     Mexican     842.97
            Indian      546.68
Gold        Mexican     163.93
Name: total_amount, dtype: float64

In [64]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


quarter
2023Q4    2477.88
2023Q1    1389.65
2023Q3     163.93
Freq: Q-DEC, Name: total_amount, dtype: float64